In [1]:
!pip install --upgrade langchain openai  -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 31.6 MB/s eta 0:00:00


In [2]:
!pip install sentence_transformers -q

In [3]:
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 6.0 MB/s eta 0:00:00
   

In [4]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (313 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123633 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [5]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from transformers import AutoModelForMaskedLM, AutoTokenizer
from torch.optim import AdamW
import torch

# Download NLTK stopwords|
nltk.download('stopwords')

# Load Dataset
data = pd.read_csv("https://raw.githubusercontent.com/Somesh-Banerjee-neu/portfolio-dataset/refs/heads/main/icliniq_medical_qa_2.csv")  # Replace with the correct dataset path
data = data.dropna()  # Remove missing values
data = data[['Question', 'Answer']]  # Use only relevant columns


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
data.shape

(19815, 2)

In [7]:
# Prepare stop words
stop_words = set(stopwords.words('english'))

#def preprocess_text(text):
#    text = text.lower()
#    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
#    text = re.sub(r'\d+', '', text)  # Remove numbers
#    tokens = text.split()
#    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
#    return " ".join(tokens)


# Apply preprocessing to questions
#data['Question'] = data['Question'].apply(preprocess_text)
#data['Answer'] = data['Answer'].apply(preprocess_text)

# Proceed with embedding generation and Pinecone upload
questions = data['Question'].tolist()
answers = data['Answer'].tolist()

In [13]:
!pip show langchain pinecone-client
!pip install pinecone-client
!pip install -U langchain-community

Name: langchain
Version: 0.3.11
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: pinecone-client
Version: 5.0.1
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: certifi, pinecone-plugin-inference, pinecone-plugin-interface, tqdm, typing-extensions, urllib3
Required-by: 
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.0 MB/s eta 0:00:00


In [14]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
import os
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Pinecone as LangchainPinecone

In [16]:
# Initialize Pinecone
api_key = "pcsk_4UnUtz_GX8H2actuQQud7t7XU5rfVeLVBZYVyyH7aVvqwA9q9sFoJenB7eVE7cihSHMKQe"  # Replace with your actual Pinecone API key
pc = Pinecone(api_key=api_key)

# Define the index name
index_name = "mediqueryx"

# Create or connect to the Pinecone index
index = pc.Index(index_name)

# Check if the index exists, and create it if it doesn't
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,  # Adjust this based on your embedding model's output dimension
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # Adjust the region as needed
        )
    )


# Initialize embeddings
embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

<ipython-input-16-f4bd8053e4cc>:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public m

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
import uuid  # For generating unique IDs

In [18]:
import uuid  # For generating unique IDs

# Loop through the questions and answers
vectors = []
for question, answer in zip(questions, answers):
    # Generate embedding for the question
    embedding = embeddings.embed_query(question)

    # Create a unique ID for each vector
    vector_id = str(uuid.uuid4())

    # Append vector to the list
    vectors.append({
        "id": vector_id,
        "values": embedding,  # Embedding values
        "metadata": {
            "question": question,
            "answer": answer
        }
    })

In [19]:
batch_size = 100  # Adjust this size as needed
for i in range(0, len(vectors), batch_size):
    batch = vectors[i:i + batch_size]
    index.upsert(vectors=batch)
    print(f"Upserted batch {i // batch_size + 1} of {len(vectors) // batch_size + 1}")

Upserted batch 1 of 199
Upserted batch 2 of 199
Upserted batch 3 of 199
Upserted batch 4 of 199
Upserted batch 5 of 199
Upserted batch 6 of 199
Upserted batch 7 of 199
Upserted batch 8 of 199
Upserted batch 9 of 199
Upserted batch 10 of 199
Upserted batch 11 of 199
Upserted batch 12 of 199
Upserted batch 13 of 199
Upserted batch 14 of 199
Upserted batch 15 of 199
Upserted batch 16 of 199
Upserted batch 17 of 199
Upserted batch 18 of 199
Upserted batch 19 of 199
Upserted batch 20 of 199
Upserted batch 21 of 199
Upserted batch 22 of 199
Upserted batch 23 of 199
Upserted batch 24 of 199
Upserted batch 25 of 199
Upserted batch 26 of 199
Upserted batch 27 of 199
Upserted batch 28 of 199
Upserted batch 29 of 199
Upserted batch 30 of 199
Upserted batch 31 of 199
Upserted batch 32 of 199
Upserted batch 33 of 199
Upserted batch 34 of 199
Upserted batch 35 of 199
Upserted batch 36 of 199
Upserted batch 37 of 199
Upserted batch 38 of 199
Upserted batch 39 of 199
Upserted batch 40 of 199
Upserted 

In [20]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import SentenceTransformerEmbeddings

In [22]:
def get_similar_docs(query, k=1, score=False):
    """
    Retrieve similar documents from the Pinecone index based on a query.

    Args:
        query (str): The query string to search for similar documents.
        k (int): Number of similar documents to retrieve.
        score (bool): Whether to include scores in the results.

    Returns:
        list: A list of similar documents (with scores if score=True).
    """
    # Convert the query into a vector using the embeddings model
    query_vector = embeddings.embed_query(query)

    # Query Pinecone for similar documents
    results = index.query(
        vector=query_vector,
        top_k=k,
        include_metadata=True  # Include metadata in results
    )

    # Extract results based on whether scores are needed
    if score:
        similar_docs = [
            {"metadata": match["metadata"], "score": match["score"]}
            for match in results["matches"]
        ]
    else:
        similar_docs = [match["metadata"] for match in results["matches"]]

    return similar_docs

In [23]:
# Example Usage:
query = (
    "How can i reduce weight give me tips"
)
similar_docs = get_similar_docs(query, k=1, score=True)

# Print the results
print(similar_docs)

[{'metadata': {'answer': 'A simple formula is followed to reduce weight. You should eat foods with reduced calories. You should take fewer calories when compared to the calories burned through the exercise. Do a workout for 45 minutes every day. You should have a good sleep for seven to eight hours a day. Stop eating junk foods. Avoid drinking alcohol if you have the habit. Take a handful of nuts every day. Avoid having stress.', 'question': 'I am 21 years old. I want to reduce my weight soon. My height is 5 feet, and my weight is 74 kilograms. What are the procedures followed to reduce weight?'}, 'score': 0.761011124}]
